# Initialization

In [2]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [4]:
items = pd.read_parquet("./goodsread/items.par")
events = pd.read_parquet("./goodsread/events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [5]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [5]:
cold_users = set(users_test) - set(common_users)

print(len(cold_users)) 

2365


In [6]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted",ascending=False).reset_index()

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.loc[item_popularity["avg_rating"] >= 4][0:100]
top_k_pop_items

,index,item_id,users,avg_rating,popularity_weighted
2,32387,18007564,20207,4.321275,87320.0
3,32623,18143977,19462,4.290669,83505.0
4,30695,16096824,16770,4.301014,72128.0
5,2,3,15139,4.706057,71245.0
7,3718,38447,14611,4.232770,61845.0
...,...,...,...,...,...
131,19596,2767052,4361,4.413437,19247.0
133,32835,18293427,4674,4.092640,19129.0
134,378,3636,4667,4.098564,19128.0
135,33611,18966819,4361,4.374914,19079.0


In [7]:
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], 
    on="item_id"
    ).sort_values(by="popularity_weighted",ascending=False).copy()

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


# === Знакомство: первые персональные рекомендации

In [8]:
cold_users_events_with_recs = events_test[events_test["user_id"].isin(cold_users)].merge(top_k_pop_items[["avg_rating","item_id"]], on="item_id", how="left")
cold_users_events_with_recs

,item_id,started_at,read_at,is_read,rating,is_reviewed,user_id,avg_rating
0,6900,2017-10-09,2017-10-13,True,4,False,1361610,NaN
1,12555,2017-09-21,2017-10-11,True,3,False,1361610,NaN
2,25899336,2017-09-12,2017-09-17,True,4,True,1361610,4.427261
3,21936809,2017-08-20,2017-08-24,True,4,True,1361610,NaN
4,6952,2017-09-18,2017-09-20,True,3,False,1361610,NaN
...,...,...,...,...,...,...,...,...
9667,252499,2017-09-30,2017-10-06,True,4,False,1178502,NaN
9668,51113,2017-09-25,2017-10-07,True,4,False,1253160,NaN
9669,16181775,2017-09-24,2017-09-25,True,3,False,1253160,NaN
9670,10210,2017-09-16,2017-09-24,True,5,False,1253160,NaN


In [9]:

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx][["user_id", "item_id", "rating", "avg_rating"]]
print("количество холдных пользователей прочитавших книги из top100 рекомендаций",len(cold_user_recs))
cold_user_recs.head()

количество холдных пользователей прочитавших книги из top100 рекомендаций 1912


,user_id,item_id,rating,avg_rating
2,1361610,25899336,4,4.427261
5,1338996,16096824,5,4.301014
8,1338996,18692431,5,4.071454
9,1338996,28763485,2,4.194663
15,1276025,38447,5,4.232770


In [10]:
print(len(cold_user_recs)/len(cold_users))

0.8084566596194503


In [11]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [22]:
cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Базовые подходы: коллаборативная фильтрация

sparsity= количество всех ячеек / количество пустых ячеек

 - Количество всех ячеек это количество пользователей * количество объъектов
 - количество всех оценок в таблице events - это непустое значение на пересечении пользователь-книга

In [54]:
sparsity = (1 - len(events)/(events['user_id'].nunique() * events['item_id'].nunique())) * 100
print("sparsity =",sparsity,"%")

sparsity = 99.92890618975984 %


In [8]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

In [9]:
import joblib

joblib.dump(svd_model, 'models/svd_recsys.joblib') 

['models/svd_recsys.joblib']

In [12]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [38]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

mae_rand = accuracy.mae(random_predictions)
print(mae_rand)

MAE:  1.0018
1.0017726877569562


# Факультативно

Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

In [27]:
events["count_events_with_item"] = events.groupby(["item_id"])["user_id"].transform("count")
events = events[events["count_events_with_item"] >= 3].copy()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428212 123177 120808


In [29]:
events["count_events_with_item"] = events.groupby(["item_id"])["user_id"].transform("count")
less_readers = events[events["count_events_with_item"] == 3].copy()
less_readers["user_id"][:10]

8807     1160232
27461    1274944
28863    1222949
32033    1227233
34221    1407657
34753    1063891
37581    1331968
38506    1267828
41042    1262466
53937    1397124
Name: user_id, dtype: int64

In [58]:
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)

RMSE: 0.8287
MAE:  0.6473


In [30]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events.query("user_id == @user_id")['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"]) 

In [31]:
#несколько холодных
list(cold_users)[:7]

[1073158, 1114119, 1204233, 1024010, 1294347, 1384460, 1286158]

In [33]:
# несколько тренированных
events_train['user_id'].sample(n=7).to_list()

[1365189, 1287316, 1403873, 1358849, 1334171, 1251065, 1317281]

In [37]:
rec_list = get_recommendations_svd(1317281, items, events, svd_model).merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
rec_list

,item_id,score,author,title,genre_and_votes
0,54741,4.798998,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."
1,24814,4.783015,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,"{'Sequential Art-Comics': 680, 'Humor': 381, '..."
2,7864312,4.761605,José Antonio Cotrina,La sombra de la luna (El ciclo de la luna roja...,"{'Fantasy': 20, 'Young Adult': 7}"
3,323355,4.761081,Joseph Smith Jr.,The Book of Mormon: Another Testament of Jesus...,"{'Religion': 896, 'Nonfiction': 486, 'Christia..."
4,24812,4.745999,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."


In [38]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
#user_id = events_train['user_id'].sample().iat[0]
user_id = 1317281

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1317281
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
49,Jenny Han,To All the Boys I've Loved Before (To All the ...,2016-08-27,2016-08-31,3,"{'Young Adult': 5919, 'Romance': 4965, 'Contem..."
50,Rachel Hawkins,"Rebel Belle (Rebel Belle, #1)",2016-08-24,2016-08-27,3,"{'Young Adult': 1336, 'Fantasy': 1234, 'Fantas..."
51,Renée Ahdieh,The Wrath and the Dawn (The Wrath and the Dawn...,2016-05-11,2016-05-12,4,"{'Fantasy': 5942, 'Young Adult': 3448, 'Romanc..."
52,Laini Taylor,Daughter of Smoke & Bone (Daughter of Smoke & ...,2016-07-03,2016-07-15,3,"{'Fantasy': 11681, 'Young Adult': 7110, 'Roman..."
53,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2016-11-28,2016-12-04,4,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
54,Victoria Aveyard,"Red Queen (Red Queen, #1)",2016-08-08,2016-08-10,4,"{'Fantasy': 10750, 'Young Adult': 6597, 'Scien..."
55,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2017-02-16,2017-02-20,5,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
56,"Amie Kaufman, Meagan Spooner","Their Fractured Light (Starbound, #3)",2016-04-27,2016-04-28,4,"{'Science Fiction': 576, 'Young Adult': 498, '..."
57,Cassandra Clare,"Clockwork Angel (The Infernal Devices, #1)",2016-08-08,2016-08-21,4,"{'Fantasy': 10151, 'Young Adult': 7424, 'Fanta..."
58,"Morgan Matson, Морган Мэтсон",The Unexpected Everything,2016-06-07,2016-06-10,5,"{'Contemporary': 1466, 'Young Adult': 1244, 'R..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,54741,4.798998,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."
1,24814,4.783015,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,"{'Sequential Art-Comics': 680, 'Humor': 381, '..."
2,7864312,4.761605,José Antonio Cotrina,La sombra de la luna (El ciclo de la luna roja...,"{'Fantasy': 20, 'Young Adult': 7}"
3,323355,4.761081,Joseph Smith Jr.,The Book of Mormon: Another Testament of Jesus...,"{'Religion': 896, 'Nonfiction': 486, 'Christia..."
4,24812,4.745999,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."


# === Базовые подходы: контентные рекомендации

In [9]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] =  item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_1896/4200202690.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_1896/4200202690.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_1896/4200202690.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [44]:
events_train['item_id_enc'].max()

43304

In [50]:
events_train['user_id'].nunique() * events_train['item_id'].nunique() / 1024**3

15.26221003383398

In [10]:
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8) 

In [56]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3 

0.26360040064901114

In [57]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

/home/mle-user/mle_projects/mle-recsys-start/.venv_recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/.venv_recsys/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:08<00:00,  3.78s/it]


In [89]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations 

In [119]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]
print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes","item_id"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder,include_seen = True)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
user_recommendations["seen"] = [  any(user_history["item_id"] == i)  for i in user_recommendations["item_id"]]
display(user_recommendations)

user_id: 1180851
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes,item_id
70,Stephen King,Gerald's Game,2017-05-06,2017-05-25,5,"{'Horror': 2871, 'Fiction': 1032, 'Thriller': ...",32692
71,"Stephen King, William Olivier Desmond, Maria T...",Dreamcatcher,2011-12-30,2016-04-27,5,"{'Horror': 2966, 'Fiction': 980, 'Science Fict...",11570
72,Stephen King,Cell,2017-06-05,2017-06-07,4,"{'Horror': 3732, 'Fiction': 1179, 'Horror-Zomb...",10567
73,Stephen King,The Tommyknockers,2016-04-30,2016-05-13,4,"{'Horror': 3264, 'Fiction': 1039, 'Science Fic...",17660
74,Stephen King,The Gunslinger,2015-12-04,2015-12-07,5,"{'Fantasy': 11423, 'Fiction': 4008, 'Horror': ...",43615
75,Stephen King,The Shining,2016-06-06,2016-06-08,5,"{'Horror': 14597, 'Fiction': 5659, 'Thriller':...",11588
76,Stephen King,Misery,2017-05-15,2017-05-16,5,"{'Horror': 7772, 'Fiction': 2953, 'Thriller': ...",10614
77,"J.R.R. Tolkien, Peter S. Beagle","The Two Towers (The Lord of the Rings, #2)",2016-12-27,2017-01-01,5,"{'Fantasy': 28091, 'Fiction': 6763, 'Classics'...",15241
78,J.K. Rowling,Harry Potter and the Deathly Hallows (Harry Po...,2017-06-18,2017-06-20,5,"{'Fantasy': 46667, 'Young Adult': 15403, 'Fict...",136251
79,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,2016-12-19,2016-12-27,5,"{'Fantasy': 38907, 'Classics': 10145, 'Fiction...",34


Рекомендации


,item_id_enc,score,item_id,author,title,genre_and_votes,seen
0,33950,1.397805,16130549,Stephen King,Doctor Sleep,"{'Horror': 5287, 'Fiction': 1848, 'Thriller': ...",False
1,28851,1.252845,10644930,Stephen King,11/22/63,"{'Fiction': 4864, 'Historical-Historical Ficti...",False
2,37068,1.221485,18775247,Stephen King,"Mr. Mercedes (Bill Hodges Trilogy, #1)","{'Fiction': 1854, 'Thriller': 1706, 'Horror': ...",False
3,1391,1.189910,11588,Stephen King,The Shining,"{'Horror': 14597, 'Fiction': 5659, 'Thriller':...",True
4,32205,1.083562,13596166,"Stephen King, Glen Orbik",Joyland,"{'Horror': 1392, 'Mystery': 1308, 'Fiction': 1...",False


In [120]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100) 

In [121]:
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [122]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("goodsread/als_recommendations.parquet") 

In [123]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
) 

In [124]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg 

In [125]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [126]:
print(ndcg_at_5_scores.mean()) 

0.9756221596154557


In [131]:
ndcg_at_5_scores.groupby("user_id").count()

user_id
1000006    1
1000007    0
1000019    0
1000020    0
1000023    1
          ..
1430558    0
1430569    0
1430573    0
1430578    1
1430584    0
Length: 48121, dtype: int64

преобразуем значения в genre_and_votes из текстового представления в тип в Python

In [4]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [5]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10) 

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


Функция строит матрицу вида «книга-жанр». матрицы содержат множество пропусков, то есть являются разреженными. Чтобы сэкономить память при работе с ними, используем sparse-формат.

In [6]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [11]:
items = items.sort_values(by="item_id_enc")

In [12]:
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [14]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres,user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [15]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0)) 

In [16]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5) 

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.185241
38,Classics,3414934,0.103879
25,Fantasy,6850060,0.072447
5,Nonfiction,1737406,0.050865
24,Science Fiction,1218917,0.040920


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

similarity_scores

array([0.50714765, 0.49770536, 0.52433355, ..., 0.64894348, 0.03112624,
       0.29805756])

In [18]:
# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = similarity_scores.argsort()[::-1][:k]

In [19]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"
39408,"Paulo Coelho, Alan R. Clarke, James Noel Smith",The Alchemist,"{'Fiction': 14023, 'Classics': 5787, 'Fantasy': 3289, 'Philosophy': 2759}"


# === Базовые подходы: валидация

In [20]:
als_recommendations =  pd.read_parquet("goodsread/als_recommendations.parquet")

##### Cобытия в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
 - gt (ground truth): объект есть в тестовой выборке;
 - pr (predicted): объект есть в рекомендациях.

##### Теперь разметим признаки бинарной классификации:
 - TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
 - FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
 - FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [25]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [21]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5)

events_recs_for_binary_metrics

/tmp/ipykernel_1896/3048627241.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


,user_id,item_id,gt,score,pr,tp,fp,fn
0,1196635,18467802,True,NaN,False,False,False,True
1,1188739,10799,True,NaN,False,False,False,True
2,1001879,13206828,True,NaN,False,False,False,True
3,1001879,13206900,True,NaN,False,False,False,True
4,1001879,13206760,True,NaN,False,False,False,True
...,...,...,...,...,...,...,...,...
1030116,1430584,29056083,False,0.462653,True,False,True,False
1030117,1430584,17167166,False,0.456820,True,False,True,False
1030118,1430584,17927395,False,0.443951,True,False,True,False
1030119,1430584,16096824,False,0.443670,True,False,True,False


In [24]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall


In [22]:
compute_cls_metrics(events_recs_for_binary_metrics)

(0.007657661313228862, 0.014173346821289545)

In [23]:
events_recs_for_binary_metrics_k10 = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=10)

compute_cls_metrics(events_recs_for_binary_metrics_k10)


/tmp/ipykernel_1896/3048627241.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


(0.008682632300788003, 0.03116118372682361)

# === Двухстадийный подход: метрики

In [24]:
als_recommendations

,user_id,item_id,score
0,1000000,3,0.960111
1,1000000,15881,0.878722
2,1000000,5,0.855399
3,1000000,6,0.826737
4,1000000,2,0.789068
...,...,...,...
43058095,1430584,13206760,0.098786
43058096,1430584,3228917,0.098126
43058097,1430584,5129,0.097948
43058098,1430584,8520610,0.097732


In [25]:
cov_items = als_recommendations["item_id"].nunique()/len(items)
print(f"{cov_items:.2f}") 

0.09


In [27]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train, on=["user_id", "item_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by="score",ascending=False)
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
novelty_5.mean()

/tmp/ipykernel_1896/2524000248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True


0.6077012223019596

# === Двухстадийный подход: модель

In [6]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()
events_labels["user_id"].nunique()

99849

In [31]:
events_labels.head(3)

,item_id,started_at,read_at,is_read,rating,is_reviewed,user_id,user_id_enc,item_id_enc,gt
84,18467802,2017-09-01,2017-09-22,True,1,False,1196635,196635,36588,True
257,10799,2017-08-06,2017-10-14,True,3,False,1188739,188739,1262,True
273,18658071,2017-09-11,2017-09-15,True,2,True,1001879,1879,36848,True


In [7]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("goodsread/candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("goodsread/candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer") 
len(candidates)

82993094

In [30]:
candidates.head(3)

,user_id,item_id,als_score,cnt_score
0,1000000,3,0.972557,0.920225
1,1000000,15881,0.890201,0.905740
2,1000000,5,0.865850,0.918026


In [8]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"],how="left")
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

In [10]:
# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

len(candidates_for_train)

213708

In [11]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 77.9ms	remaining: 1m 17s
100:	learn: 0.5118959	total: 1.93s	remaining: 17.2s
200:	learn: 0.5111710	total: 3.87s	remaining: 15.4s
300:	learn: 0.5105208	total: 5.83s	remaining: 13.5s
400:	learn: 0.5100174	total: 7.74s	remaining: 11.6s
500:	learn: 0.5095747	total: 9.64s	remaining: 9.6s
600:	learn: 0.5091600	total: 11.5s	remaining: 7.66s
700:	learn: 0.5087803	total: 13.5s	remaining: 5.77s
800:	learn: 0.5084220	total: 15.5s	remaining: 3.85s
900:	learn: 0.5080930	total: 17.4s	remaining: 1.91s
999:	learn: 0.5078081	total: 19.3s	remaining: 0us


In [17]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("goodsread/candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("goodsread/candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
    )


In [18]:

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank)) 

14517152


In [21]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [22]:
len(final_recommendations)

7519400

In [26]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}") 

Common users: 75194
precision: 0.006, recall: 0.015


# === Двухстадийный подход: построение признаков